# Compute P-/S-wave travel Times

In this notebook, we compute the table of P-/S-wave travel times from a 3D grid of potential source locations to every seismic station. This travel time table is necessary for backprojection and for earthquake location with `NLLoc`.

This tutorial uses the `pykonal` package to compute travel times in a given velocity model (see `pykonal` documentation at [https://github.com/malcolmw/pykonal](https://github.com/malcolmw/pykonal)). Please acknowledge [White et al. (2020)](https://pubs.geoscienceworld.org/ssa/srl/article-abstract/91/4/2378/586804/PyKonal-A-Python-Package-for-Solving-the-Eikonal?redirectedFrom=fulltext) if using `pykonal`.

In [1]:
import BPMF
import h5py as h5
import numpy as np
import os
import pykonal
import pandas as pd
import sys


from BPMF import NLLoc_utils
from obspy import UTCDateTime as udt
from scipy.interpolate import interp1d, LinearNDInterpolator
from time import time as give_time

## Read the velocity model from Karabulut et al. 2011

In [2]:
# we assume that you are running this notebook from Seismic_BPMF/notebooks/ and the velocity model
# is stored at Seismic_BPMF/data/
FILEPATH_VEL_MODEL = os.path.join(os.pardir, "data", "velocity_model_Karabulut2011.csv")

We read the velocity model with `pandas`. Not all columns of the csv files are necessary for our purpose. The velocity model is a 1D layered model and the `depth` column gives the top of each layer in meters (positive downward). The `P` and `S` columns are the wave velocities in m/s.

In [27]:
# read the velocity model with pandas
velocity_model = pd.read_csv(
    FILEPATH_VEL_MODEL, 
    usecols=[1, 2, 4],
    names=["depth", "P", "S"],
    skiprows=1,
    )
velocity_model.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
depth,-2000.0,0.0,1000.0,2000.0,3000.0,4000.0,5000.0,6000.0,8000.0,10000.0,12000.0,14000.0,15000.0,20000.0,22000.0,25000.0,32000.0,77000.0
P,2900.0,3000.0,5600.0,5700.0,5800.0,5900.0,5950.0,6050.0,6100.0,6150.0,6200.0,6250.0,6300.0,6400.0,6500.0,6700.0,8000.0,8045.0
S,1670.0,1900.0,3150.0,3210.0,3260.0,3410.0,3420.0,3440.0,3480.0,3560.0,3590.0,3610.0,3630.0,3660.0,3780.0,3850.0,4650.0,4650.0


## Build a 3D Grid of P-/S-wave Velocities

Even though the model used here is a 1D model, `pykonal` works with a 3D model. Therefore, we first need to define the velocity model on a 3D spatial grid.

In [22]:
# define constant to go from degrees to radians
DEG_TO_RAD = np.pi / 180.
# define the geographical extent of the 3D grid
LON_MIN_DEG = 30.20
LON_MAX_DEG = 30.45
LAT_MIN_DEG = 40.60
LAT_MAX_DEG = 40.75
DEP_MIN_KM = -2.0
DEP_MAX_KM = 30.
# define the grid spacing
D_LON_DEG = 0.01 
D_LAT_DEG = 0.01
D_LON_RAD = D_LON_DEG * DEG_TO_RAD
D_LAT_RAD = D_LAT_DEG * DEG_TO_RAD
D_DEP_KM = 0.5

In [28]:
# we will be using pykonal in spherical coordinates with radial coordinates (depth) in km
# therefore, we first convert all meters in kilometers
velocity_model["depth"] /= 1000.0
velocity_model["P"] /= 1000.0
velocity_model["S"] /= 1000.0


In [29]:
# to expand the 1D layered velocity model onto a 3D grid,
# we build an interpolator that gives the velocity at any point in depth
# to preserve the layered structure in the interpolated model, we need to
# add new rows to the 1D model to keep the velocity discontinuities

# new depths just below the existing depths
new_depths = velocity_model["depth"][1:] - 0.00001
# P-wave velocity interpolator
new_vP = velocity_model["P"][:-1]
interpolator_P = interp1d(
    np.hstack((velocity_model["depth"], new_depths)),
    np.hstack((velocity_model["P"], new_vP)),
)
# S-wave velocity interpolator
new_vS = velocity_model["S"][:-1]
interpolator_S = interp1d(
    np.hstack((velocity_model["depth"], new_depths)),
    np.hstack((velocity_model["S"], new_vS)),
)

The spherical coordinate system used by `pykonal` specifies a point position with $(r, \theta, \varphi)$:
- $r$: Distance from center or Earth in km (= decreasing depth).
- $\theta$: Polar angle in radians (= co-latitude or, equivalently, decreasing latitude).
- $\varphi$: Azimuthal angle in radians (= longitude).

Thus, the 3D grid must be built such that $r$, $\theta$ and $\varphi$ increase with increasing index. Axis 0 is decreasing depth, axis 1 is decreasing latitude and axis 2 is increasing longitude.

In [23]:
# make sure the user-specified ends of the grid are included in the arange vectors
longitudes = np.arange(LON_MIN_DEG, LON_MAX_DEG+D_LON_DEG/2., D_LON_DEG)
latitudes = np.arange(LAT_MAX_DEG, LAT_MIN_DEG-D_LAT_DEG/2., -D_LAT_DEG)
depths = np.arange(DEP_MAX_KM, DEP_MIN_KM-D_DEP_KM/2., -D_DEP_KM)

print("Increasing longitudes: ", longitudes)
print("Decreasing latitudes: ", latitudes)
print("Decreasing depths: ", depths)

# get the number of points in each direction
n_longitudes = len(longitudes)
n_latitudes = len(latitudes)
n_depths = len(depths)

Increasing longitudes:  [30.2  30.21 30.22 30.23 30.24 30.25 30.26 30.27 30.28 30.29 30.3  30.31
 30.32 30.33 30.34 30.35 30.36 30.37 30.38 30.39 30.4  30.41 30.42 30.43
 30.44 30.45]
Decreasing latitudes:  [40.75 40.74 40.73 40.72 40.71 40.7  40.69 40.68 40.67 40.66 40.65 40.64
 40.63 40.62 40.61 40.6 ]
Decreasing depths:  [30.  29.5 29.  28.5 28.  27.5 27.  26.5 26.  25.5 25.  24.5 24.  23.5
 23.  22.5 22.  21.5 21.  20.5 20.  19.5 19.  18.5 18.  17.5 17.  16.5
 16.  15.5 15.  14.5 14.  13.5 13.  12.5 12.  11.5 11.  10.5 10.   9.5
  9.   8.5  8.   7.5  7.   6.5  6.   5.5  5.   4.5  4.   3.5  3.   2.5
  2.   1.5  1.   0.5  0.  -0.5 -1.  -1.5 -2. ]


In [31]:
# use the previously defined interpolators to compute the P-/S-wave velocity
# at each point of the 3D spherical coordinate grid defined by depths, latitudes, longitudes (see above cell)
vP_grid = np.zeros((n_depths, n_latitudes, n_longitudes), dtype=np.float32)
vS_grid = np.zeros((n_depths, n_latitudes, n_longitudes), dtype=np.float32)
for i in range(n_depths):
    vP_grid[i, :, :] = interpolator_P(depths[i])
    vS_grid[i, :, :] = interpolator_S(depths[i])

depths_g, latitudes_g, longitudes_g = np.meshgrid(
    depths, latitudes, longitudes, indexing="ij"
)

# keep velocities and grid point coordinates in a dictionary for later use
grid = {}
grid["vP"] = vP_grid
grid["vS"] = vS_grid
grid["longitude"] = longitudes_g
grid["latitude"] = latitudes_g
grid["depth"] = depths_g

The 3D grid is ready! This piece of code can easily be adapted for using a 2D or 3D input velocity model. Keep in mind that, in most cases, even a 3D velocity model will have to be re-ordered to satisfy `pykonal` conventions.

## Compute the P-/S-wave travel times